<a href="https://colab.research.google.com/github/spentaur/DS-Unit-2-Applied-Modeling/blob/master/Copy_of_Unit2ProjectFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load in some packages


*   [ta](https://github.com/bukosabino/ta)
*   [eli5](https://eli5.readthedocs.io/en/latest/overview.html)
*   [bayesian-optimization](https://github.com/fmfn/BayesianOptimization)
*   [LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
*   [h2o4gpu](https://github.com/h2oai/h2o4gpu)

In [2]:
%%sh
pip install ta
pip install eli5
pip install bayesian-optimization
pip install pandas_market_calendars
pip install pytz

# LightGBM
# git clone --recursive https://github.com/microsoft/LightGBM ; cd LightGBM
# mkdir build ; cd build
# cmake -DUSE_GPU=1 ..
# make -j4

# h2o4gpu
export CUDA_HOME=/usr/local/cuda 
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64/:$CUDA_HOME/lib/:$CUDA_HOME/extras/CUPTI/lib64
sudo apt-get install libopenblas-dev pbzip2
wget https://s3.amazonaws.com/h2o-release/h2o4gpu/releases/stable/ai/h2o/h2o4gpu/0.3-cuda10/h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl
pip install h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl

  Created wheel for ta: filename=ta-0.4.5-cp36-none-any.whl size=16698 sha256=09570c091428083eb2b3d9cebb80096f024068d9fae34c2465145d35778b506c
  Stored in directory: /root/.cache/pip/wheels/de/ac/09/28855b628633d42061f670c59df6877a1dbdf70d04c985bc18
Successfully built ta
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=aae4561e9b56d0813cf830d92e66afbbf3aa484c16c8856c9f37b513180aa7be
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization
Reading package lists...
Building dependency tree...
Reading state information...
libopenblas-dev is already the newest version (0.2.20+ds-4).
The following NEW packages will be installed:
  pbzip2
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 37.1 kB of archives.
After this operation, 95.2 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubun

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
--2019-10-21 03:34:16--  https://s3.amazonaws.com/h2o-release/h2o4gpu/releases/stable/ai/h2o/h2o4gpu/0.3-cuda10/h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.141
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 366174349 (349M) [application/zip]
Saving to: ‘h2o4gpu-0.3.2-cp36-cp36m-linux_x86_64.whl’

     0K .......... .......... .......... .......... ..........  0%  811K 7m21s
    50K .......... .....

In [6]:
pip install pytz --upgrade

Requirement already up-to-date: pytz in /usr/local/lib/python3.6/dist-packages (2019.3)


In [8]:
pip install pytz

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import scipy.stats as st
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline
from sklearn.model_selection import train_test_split
from ta import *
from scipy.stats import ttest_ind
from scipy import stats
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import SelectFromModel
import lightgbm as lgb
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy import stats
import pandas_datareader.data as web
from datetime import datetime
import h2o4gpu
from bayes_opt import BayesianOptimization
from sklearn.feature_selection import SelectFromModel
from dateutil.rrule import *
import pandas_market_calendars as mcal
import time
from sklearn.impute import SimpleImputer

# Change some pandas defaults

In [0]:
def start():
    options = {
        'display': {
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 100,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 4,
            'show_dimensions': False
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }
    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+
start()

# Load in historical data for the SPY ETF

In [11]:
spy = web.DataReader("SPY", 'yahoo').reset_index()
spy.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2010-01-04,113.39,111.51,112.37,113.33,1.1894e+08,93.2447
1,2010-01-05,113.68,112.85,113.26,113.63,1.1158e+08,93.4915
2,2010-01-06,113.99,113.43,113.52,113.71,1.1607e+08,93.5573
3,2010-01-07,114.33,113.18,113.50,114.19,1.3109e+08,93.9523
4,2010-01-08,114.62,113.66,113.89,114.57,1.2640e+08,94.2649


In [12]:
spy.tail()

,Date,High,Low,Open,Close,Volume,Adj Close
2461,2019-10-14,296.67,295.57,295.93,295.95,4.0395e+07,295.95
2462,2019-10-15,299.70,296.97,297.10,298.88,4.6754e+07,298.88
2463,2019-10-16,299.16,297.92,298.37,298.40,4.8928e+07,298.40
2464,2019-10-17,300.24,298.52,299.68,299.28,4.5737e+07,299.28
2465,2019-10-18,299.40,297.03,298.69,297.97,6.4304e+07,297.97


# get target

In [0]:
def get_target(data, days_out):
    data = data.copy()
    # get the close price X days out
    data['x_close'] = data['Close'].shift(days_out)
    # get the % return 20 days out
    data['x_return'] = (((data['x_close'] / data['Close']) - 1)  * 100)
    # is return greater than 1%
    data['Up'] = data['x_return'] > 1
    # price at 1% return
    data['1% Price'] = data['Close'] * 1.01
    # get the date 20 days out
    nyse = mcal.get_calendar('NYSE')
    nyse_schedule = nyse.schedule(start_date='2010-01-04', end_date='2022-12-31').reset_index()
    nyse_schedule.rename(columns={'index': 'Date'}, inplace=True)
    nyse_schedule = spy.merge(nyse_schedule, how = "outer")
    data['x_days_out'] = nyse_schedule['Date'].shift(days_out)

    return data[['Date','Close', 'x_close', 'x_return', '1% Price', 'Up', 'x_days_out']]

In [14]:
days_out = -20
target = get_target(spy, days_out)
target.tail(30)

,Date,Close,x_close,x_return,1% Price,Up,x_days_out
2436,2019-09-09,298.20,293.08,-1.7170,301.1820,False,2019-10-07
2437,2019-09-10,298.13,288.53,-3.2201,301.1113,False,2019-10-08
2438,2019-09-11,300.25,291.27,-2.9908,303.2525,False,2019-10-09
2439,2019-09-12,301.29,293.24,-2.6719,304.3029,False,2019-10-10
2440,2019-09-13,301.09,296.28,-1.5975,304.1009,False,2019-10-11
2441,2019-09-16,300.16,295.95,-1.4026,303.1616,False,2019-10-14
2442,2019-09-17,300.92,298.88,-0.6779,303.9292,False,2019-10-15
2443,2019-09-18,301.10,298.40,-0.8967,304.1110,False,2019-10-16
2444,2019-09-19,301.08,299.28,-0.5978,304.0908,False,2019-10-17
2445,2019-09-20,298.28,297.97,-0.1039,301.2628,False,2019-10-18


# Wrangle

In [0]:
def wrangle(data, drop_date=True):
    data = data.copy()

    # change Date to datetime type
    data['Date'] = pd.to_datetime(data['Date'])
    # change date to index
    data.index = pd.DatetimeIndex(data['Date'])

    # not sure if this is needed, but since i'm doing just a basic train test
    # split i want to make sure it's sorted
    data = data.sort_index()

    # add some time features
    data['Day of Week'] = data['Date'].dt.weekday
    data['Year'] = data['Date'].dt.year
    data['Month'] = data['Date'].dt.month
    data['Quarter'] = data['Date'].dt.quarter
    data['Week'] = data['Date'].dt.quarter

    # drop the actual date column
    if drop_date:
        data = data.drop('Date', axis=1)
        
    # Daily High - Daily Low
    data['High - Low'] = data['High'] - data['Low']
    data['High - Low as % of Close'] = data['High - Low'] / data['Close']
    # High - Close
    data['High - Close'] = data['High'] - data['Close']
    data['High - Close as % of Close'] = data['High - Close'] / data['Close']
    # High - Open
    data['High - Open'] = data['High'] - data['Open']
    data['High - Open as % of Close'] = data['High - Open'] / data['Close']
    # Today Open - Yesterday Close
    data['Open - Close'] = data['Open'] - data['Close'].shift(1)
    data['Open - Close as % of Close'] = data['Open - Close'] / data['Close']


    # add all the technical analysis features from ta
    data = add_all_ta_features(data, "Open", "High", "Low", "Close", 
                               "Volume", fillna=False)
    
    # delete the columns that are over 50% missing
    perc_missing = data.isna().sum() / len(data)
    data = data.drop(perc_missing[perc_missing > .50].index.tolist(), axis=1)

    # some of the features use ~45 day rolling averages and stuff
    # so i'll just skip the first 45 days
    # and then i'll also remove the days that wont have the target
    # since it hasn't happened yet (20 days out)
    data = data.iloc[45:]

    # i'm just going to divide every feature by every other feature
    # there's gotta be a pattern in the data somewhere right
    div_features = pd.DataFrame()
    cols = data.select_dtypes('number').columns.tolist()
    n = len(cols)
    for idx, first in enumerate(cols):
        x = (n-idx) - 1
        for second in cols[:x]:
            div_features['{} / {}'.format(first, second)] = data[first] / data[second]

    data = pd.concat([data, div_features], axis=1)
    
    # now i gotta do something about all the inf values i just created
    data = data.replace([np.inf, -np.inf], np.nan)
    data = data.interpolate(method='time')
    data = data.fillna(data.mean())

    perc_missing = data.isna().sum() / len(data)
    data = data.drop(perc_missing[perc_missing > .50].index.tolist(), axis=1)
    
    # done
    return data

In [16]:
# wrangle only the data for which i'd have the target
df = wrangle(spy.iloc[:days_out])
df.shape

/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: divide by zero encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: divide by zero encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:176: RuntimeWarning: invalid value encountered in subtract
  dx = 100 * np.abs((dip - din) / (dip + din))


(2401, 2700)

# Fit the model

In [17]:
target_feature = 'Up'
X = df
y = target[target_feature].iloc[45:days_out]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.015, random_state=42, shuffle=False)
X_train.shape, X_test.shape

((2364, 2700), (37, 2700))

In [18]:
clf = ''
clf = RandomForestClassifier(n_estimators=800,n_jobs=-1, random_state=42)

clf.fit(X_train, y_train)
ypred = clf.predict(X_test)
accuracy_score(y_test, ypred)
# 0.8918918918918919

0.8918918918918919

In [19]:
y_test.value_counts(normalize=True)

True     0.5135
False    0.4865
Name: Up, dtype: float64

# Select Best Features

In [0]:
features = pd.DataFrame({'col': X_train.columns, 'importance': clf.feature_importances_}).sort_values('importance', ascending=False)
features.head(30)

,col,importance
1120,High - Open as % of C...,0.0050
890,High - Low as % of Cl...,0.0040
1065,High - Open / volatil...,0.0036
20,volume_obv,0.0032
1626,volatility_atr / vola...,0.0030
1283,volume_adi / volatili...,0.0029
1443,volume_fi / volatilit...,0.0029
1175,Open - Close / volati...,0.0028
1121,High - Open as % of C...,0.0027
1229,Open - Close as % of ...,0.0027


In [0]:
feature_names = features['col'].tolist()[:500]

In [0]:
len(feature_names)

500

In [22]:
clf = ''
clf = h2o4gpu.RandomForestClassifier(n_estimators=800,n_jobs=-1, random_state=42)

clf.fit(X_train[feature_names], y_train)
ypred = clf.predict(X_test[feature_names])
accuracy_score(y_test, ypred)
# 0.918918918918919

0.918918918918919

In [0]:
# base_score = 0.918918918918919
# feature_scores = {}
# count = 0
# for idx in range(len(feature_names) - 1, 0, -1):
#     start = time.time()
#     count += 1
#     feature_removed = feature_names[idx]
#     feats = set(feature_names) - set([feature_removed])
#     print(feature_removed)
#     print(len(feats))
#     clf = ''
#     clf = h2o4gpu.RandomForestClassifier(n_estimators=800,n_jobs=-1, random_state=42)

#     clf.fit(X_train[feats], y_train)
#     ypred = clf.predict(X_test[feats])
#     score = accuracy_score(y_test, ypred)
#     feature_scores[feature_removed] = base_score - score
#     if base_score - score > 0:
#         print('removing feature caused score to go down by', base_score - score)
#     if base_score - score < 0:
#         print('removing feature caused score to go up by', base_score - score)
#     print(count, '/', len(feature_names))
#     print(score)
#     print(time.time() - start)
#     print('\n')
    

Month / trend_vortex_ind_pos
499
removing feature caused score to go down by 0.05405405405405406
1 / 500
0.8648648648648649
9.181135654449463


Open - Close as % of Close / volatility_dchi
499
removing feature caused score to go down by 0.08108108108108114
2 / 500
0.8378378378378378
9.06017255783081


volatility_atr / Low
499
removing feature caused score to go down by 0.1351351351351352
3 / 500
0.7837837837837838
9.055663347244263


volume_obv / High - Low
499
removing feature caused score to go down by 0.18918918918918926
4 / 500
0.7297297297297297
9.033926963806152


trend_macd / Open
499
removing feature caused score to go down by 0.16216216216216217
5 / 500
0.7567567567567568
9.036326885223389


trend_kst / Close
499
removing feature caused score to go down by 0.05405405405405406
6 / 500
0.8648648648648649
9.058873653411865


Week / trend_trix
499
removing feature caused score to go down by 0.10810810810810811
7 / 500
0.8108108108108109
9.051328897476196


Low / momentum_tsi
499
r

In [0]:
feature_scores

{'Adj Close': 0.05405405405405406,
 'Adj Close / Month': 0.027027027027027084,
 'Adj Close / Quarter': 0.05405405405405406,
 'Adj Close / Week': 0.05405405405405406,
 'Adj Close / Year': 0.10810810810810811,
 'Adj Close / momentum_tsi': 0.08108108108108114,
 'Adj Close / trend_ichimoku_b': 0.027027027027027084,
 'Adj Close / trend_mass_index': 0.18918918918918926,
 'Adj Close / trend_visual_ichimoku_a': -0.05405405405405406,
 'Adj Close / trend_visual_ichimoku_b': 0.0,
 'Adj Close / volatility_bbhi': 0.05405405405405406,
 'Adj Close / volatility_bbli': 0.1351351351351352,
 'Adj Close / volatility_dchi': -0.05405405405405406,
 'Adj Close / volatility_dcli': 0.05405405405405406,
 'Adj Close / volume_nvi': 0.1351351351351352,
 'Close': 0.05405405405405406,
 'Close / Month': 0.05405405405405406,
 'Close / Quarter': 0.05405405405405406,
 'Close / Week': 0.05405405405405406,
 'Close / Year': 0.1351351351351352,
 'Close / trend_aroon_ind': 0.18918918918918926,
 'Close / trend_ichimoku_b': 0.0

In [0]:
feature_scores = {'Adj Close': 0.05405405405405406,
 'Adj Close / Month': 0.027027027027027084,
 'Adj Close / Quarter': 0.05405405405405406,
 'Adj Close / Week': 0.05405405405405406,
 'Adj Close / Year': 0.10810810810810811,
 'Adj Close / momentum_tsi': 0.08108108108108114,
 'Adj Close / trend_ichimoku_b': 0.027027027027027084,
 'Adj Close / trend_mass_index': 0.18918918918918926,
 'Adj Close / trend_visual_ichimoku_a': -0.05405405405405406,
 'Adj Close / trend_visual_ichimoku_b': 0.0,
 'Adj Close / volatility_bbhi': 0.05405405405405406,
 'Adj Close / volatility_bbli': 0.1351351351351352,
 'Adj Close / volatility_dchi': -0.05405405405405406,
 'Adj Close / volatility_dcli': 0.05405405405405406,
 'Adj Close / volume_nvi': 0.1351351351351352,
 'Close': 0.05405405405405406,
 'Close / Month': 0.05405405405405406,
 'Close / Quarter': 0.05405405405405406,
 'Close / Week': 0.05405405405405406,
 'Close / Year': 0.1351351351351352,
 'Close / trend_aroon_ind': 0.18918918918918926,
 'Close / trend_ichimoku_b': 0.0,
 'Close / trend_visual_ichimoku_a': 0.08108108108108114,
 'Close / trend_visual_ichimoku_b': -0.05405405405405406,
 'Close / volatility_atr': 0.05405405405405406,
 'Close / volatility_bbhi': 0.05405405405405406,
 'Close / volatility_dcli': 0.08108108108108114,
 'Close / volume_nvi': 0.05405405405405406,
 'Day of Week / volatility_bbhi': 0.0,
 'Day of Week / volatility_bbli': -0.027027027027026973,
 'Day of Week / volatility_dcli': 0.0,
 'High': 0.05405405405405406,
 'High - Close / High - Close as % of Close': 0.05405405405405406,
 'High - Close / volatility_bbhi': -0.027027027027026973,
 'High - Close / volatility_bbli': 0.0,
 'High - Close / volatility_dchi': 0.16216216216216217,
 'High - Close / volatility_dcli': 0.18918918918918926,
 'High - Close as % of Close / High - Close': 0.08108108108108114,
 'High - Close as % of Close / volatility_bbhi': 0.0,
 'High - Close as % of Close / volatility_bbli': 0.10810810810810811,
 'High - Close as % of Close / volatility_dchi': 0.0,
 'High - Close as % of Close / volatility_dcli': 0.027027027027027084,
 'High - Low / High - Low as % of Close': 0.05405405405405406,
 'High - Low / volatility_bbhi': -0.027027027027026973,
 'High - Low / volatility_bbli': 0.1351351351351352,
 'High - Low / volatility_dcli': 0.1351351351351352,
 'High - Low as % of Close / High - Low': 0.05405405405405406,
 'High - Low as % of Close / volatility_bbhi': 0.0,
 'High - Low as % of Close / volatility_bbli': 0.027027027027027084,
 'High - Low as % of Close / volatility_dcli': 0.18918918918918926,
 'High - Open / High - Open as % of Close': 0.05405405405405406,
 'High - Open / volatility_bbhi': 0.0,
 'High - Open / volatility_bbli': 0.1351351351351352,
 'High - Open / volatility_dchi': 0.0,
 'High - Open / volatility_dcli': 0.027027027027027084,
 'High - Open as % of Close / High - Open': 0.05405405405405406,
 'High - Open as % of Close / volatility_bbli': 0.027027027027027084,
 'High - Open as % of Close / volatility_dchi': 0.027027027027027084,
 'High - Open as % of Close / volatility_dcli': 0.05405405405405406,
 'High / Month': 0.05405405405405406,
 'High / Quarter': 0.05405405405405406,
 'High / Week': 0.05405405405405406,
 'High / Year': 0.05405405405405406,
 'High / trend_aroon_ind': 0.10810810810810811,
 'High / trend_ichimoku_b': 0.05405405405405406,
 'High / trend_visual_ichimoku_a': -0.027027027027026973,
 'High / trend_visual_ichimoku_b': -0.027027027027026973,
 'High / volatility_atr': 0.05405405405405406,
 'High / volatility_bbhi': 0.0,
 'High / volatility_bbli': 0.027027027027027084,
 'High / volatility_dchi': 0.1351351351351352,
 'High / volatility_dcli': 0.0,
 'High / volume_nvi': 0.10810810810810811,
 'Low': 0.05405405405405406,
 'Low / Month': 0.05405405405405406,
 'Low / Quarter': 0.05405405405405406,
 'Low / Week': 0.05405405405405406,
 'Low / Year': 0.0,
 'Low / momentum_tsi': 0.05405405405405406,
 'Low / trend_aroon_ind': 0.18918918918918926,
 'Low / trend_ichimoku_b': 0.05405405405405406,
 'Low / trend_mass_index': -0.027027027027026973,
 'Low / trend_visual_ichimoku_a': 0.10810810810810811,
 'Low / trend_visual_ichimoku_b': 0.08108108108108114,
 'Low / volatility_atr': 0.05405405405405406,
 'Low / volatility_bbhi': 0.05405405405405406,
 'Low / volatility_bbli': 0.05405405405405406,
 'Low / volatility_dchi': 0.05405405405405406,
 'Low / volatility_dcli': 0.16216216216216217,
 'Low / volume_nvi': 0.1351351351351352,
 'Month / Adj Close': -0.027027027027026973,
 'Month / Close': 0.05405405405405406,
 'Month / High': 0.10810810810810811,
 'Month / Low': 0.05405405405405406,
 'Month / Open': 0.05405405405405406,
 'Month / Year': 0.05405405405405406,
 'Month / trend_ema_fast': 0.05405405405405406,
 'Month / trend_ema_slow': 0.05405405405405406,
 'Month / trend_ichimoku_a': 0.1351351351351352,
 'Month / trend_ichimoku_b': 0.05405405405405406,
 'Month / trend_kst': 0.1351351351351352,
 'Month / trend_kst_sig': 0.05405405405405406,
 'Month / trend_mass_index': 0.16216216216216217,
 'Month / trend_visual_ichimoku_a': 0.10810810810810811,
 'Month / trend_visual_ichimoku_b': 0.05405405405405406,
 'Month / trend_vortex_ind_pos': 0.05405405405405406,
 'Month / volatility_atr': 0.0,
 'Month / volatility_bbh': 0.08108108108108114,
 'Month / volatility_bbhi': 0.1351351351351352,
 'Month / volatility_bbl': 0.05405405405405406,
 'Month / volatility_bbli': 0.027027027027027084,
 'Month / volatility_bbm': 0.05405405405405406,
 'Month / volatility_dch': 0.0,
 'Month / volatility_dcl': 0.05405405405405406,
 'Month / volatility_dcli': -0.027027027027026973,
 'Month / volatility_kcc': 0.05405405405405406,
 'Month / volatility_kch': 0.05405405405405406,
 'Month / volatility_kcl': 0.1351351351351352,
 'Month / volume_nvi': 0.10810810810810811,
 'Open': 0.05405405405405406,
 'Open - Close / Open - Close as % of Close': 0.05405405405405406,
 'Open - Close / volatility_bbhi': 0.027027027027027084,
 'Open - Close / volatility_bbli': 0.08108108108108114,
 'Open - Close / volatility_dchi': 0.0,
 'Open - Close / volatility_dcli': 0.027027027027027084,
 'Open - Close as % of Close / Open - Close': 0.05405405405405406,
 'Open - Close as % of Close / volatility_bbhi': 0.1351351351351352,
 'Open - Close as % of Close / volatility_bbli': 0.08108108108108114,
 'Open - Close as % of Close / volatility_dchi': 0.08108108108108114,
 'Open - Close as % of Close / volatility_dcli': 0.16216216216216217,
 'Open / Month': 0.0,
 'Open / Quarter': 0.05405405405405406,
 'Open / Week': 0.05405405405405406,
 'Open / Year': 0.05405405405405406,
 'Open / trend_aroon_ind': 0.1351351351351352,
 'Open / trend_ichimoku_b': 0.1351351351351352,
 'Open / trend_visual_ichimoku_a': 0.16216216216216217,
 'Open / trend_visual_ichimoku_b': -0.027027027027026973,
 'Open / volatility_bbhi': -0.027027027027026973,
 'Open / volatility_dcli': 0.05405405405405406,
 'Open / volume_nvi': 0.05405405405405406,
 'Quarter / Adj Close': 0.05405405405405406,
 'Quarter / Close': 0.05405405405405406,
 'Quarter / High': 0.05405405405405406,
 'Quarter / Low': 0.05405405405405406,
 'Quarter / Open': 0.05405405405405406,
 'Quarter / trend_ema_fast': 0.05405405405405406,
 'Quarter / trend_ema_slow': 0.05405405405405406,
 'Quarter / trend_ichimoku_a': 0.05405405405405406,
 'Quarter / trend_ichimoku_b': 0.05405405405405406,
 'Quarter / trend_kst_sig': 0.05405405405405406,
 'Quarter / trend_mass_index': 0.05405405405405406,
 'Quarter / trend_visual_ichimoku_a': 0.05405405405405406,
 'Quarter / trend_visual_ichimoku_b': 0.05405405405405406,
 'Quarter / volatility_atr': 0.05405405405405406,
 'Quarter / volatility_bbh': 0.05405405405405406,
 'Quarter / volatility_bbhi': 0.05405405405405406,
 'Quarter / volatility_bbl': 0.05405405405405406,
 'Quarter / volatility_bbli': 0.05405405405405406,
 'Quarter / volatility_bbm': 0.05405405405405406,
 'Quarter / volatility_dch': 0.05405405405405406,
 'Quarter / volatility_dchi': 0.05405405405405406,
 'Quarter / volatility_dcl': 0.05405405405405406,
 'Quarter / volatility_dcli': 0.05405405405405406,
 'Quarter / volatility_kcc': 0.05405405405405406,
 'Quarter / volatility_kch': 0.05405405405405406,
 'Quarter / volatility_kcl': 0.05405405405405406,
 'Quarter / volume_nvi': 0.05405405405405406,
 'Volume / volatility_bbhi': -0.027027027027026973,
 'Volume / volatility_bbli': -0.027027027027026973,
 'Volume / volatility_dcli': 0.027027027027027084,
 'Week / Adj Close': 0.05405405405405406,
 'Week / Close': 0.05405405405405406,
 'Week / High': 0.05405405405405406,
 'Week / Low': 0.05405405405405406,
 'Week / Open': 0.05405405405405406,
 'Week / trend_ema_fast': 0.05405405405405406,
 'Week / trend_ema_slow': 0.05405405405405406,
 'Week / trend_ichimoku_a': 0.05405405405405406,
 'Week / trend_ichimoku_b': 0.05405405405405406,
 'Week / trend_kst_sig': 0.05405405405405406,
 'Week / trend_trix': 0.10810810810810811,
 'Week / trend_visual_ichimoku_a': 0.05405405405405406,
 'Week / trend_visual_ichimoku_b': 0.05405405405405406,
 'Week / trend_vortex_ind_neg': 0.10810810810810811,
 'Week / volatility_atr': 0.05405405405405406,
 'Week / volatility_bbh': 0.05405405405405406,
 'Week / volatility_bbhi': 0.05405405405405406,
 'Week / volatility_bbl': 0.05405405405405406,
 'Week / volatility_bbli': 0.05405405405405406,
 'Week / volatility_bbm': 0.05405405405405406,
 'Week / volatility_dch': 0.05405405405405406,
 'Week / volatility_dchi': 0.05405405405405406,
 'Week / volatility_dcl': 0.05405405405405406,
 'Week / volatility_dcli': 0.05405405405405406,
 'Week / volatility_kcc': 0.05405405405405406,
 'Week / volatility_kch': 0.05405405405405406,
 'Week / volatility_kcl': 0.05405405405405406,
 'Week / volume_nvi': 0.05405405405405406,
 'Year / Adj Close': 0.05405405405405406,
 'Year / Close': 0.10810810810810811,
 'Year / High': -0.027027027027026973,
 'Year / Low': 0.05405405405405406,
 'Year / Month': 0.05405405405405406,
 'Year / momentum_tsi': 0.027027027027027084,
 'Year / trend_aroon_ind': 0.0,
 'Year / trend_ema_fast': 0.18918918918918926,
 'Year / trend_ema_slow': 0.10810810810810811,
 'Year / trend_ichimoku_a': 0.0,
 'Year / trend_ichimoku_b': 0.05405405405405406,
 'Year / trend_mass_index': 0.0,
 'Year / trend_visual_ichimoku_a': 0.05405405405405406,
 'Year / trend_visual_ichimoku_b': 0.05405405405405406,
 'Year / volatility_bbl': 0.05405405405405406,
 'Year / volatility_bbm': 0.1351351351351352,
 'Year / volatility_dch': 0.05405405405405406,
 'Year / volatility_dcl': 0.05405405405405406,
 'Year / volatility_kcc': 0.05405405405405406,
 'Year / volatility_kch': 0.05405405405405406,
 'Year / volatility_kcl': 0.05405405405405406,
 'Year / volume_nvi': 0.05405405405405406,
 'momentum_ao': 0.08108108108108114,
 'momentum_tsi': 0.08108108108108114,
 'momentum_tsi / Close': 0.05405405405405406,
 'momentum_tsi / High': 0.05405405405405406,
 'momentum_tsi / Low': 0.05405405405405406,
 'momentum_tsi / Open': 0.05405405405405406,
 'momentum_tsi / Year': 0.05405405405405406,
 'momentum_uo / Low': 0.027027027027027084,
 'others_cr': 0.05405405405405406,
 'trend_cci / Month': 0.16216216216216217,
 'trend_ema_fast': 0.05405405405405406,
 'trend_ema_fast / Month': 0.05405405405405406,
 'trend_ema_fast / Quarter': 0.05405405405405406,
 'trend_ema_fast / Week': 0.05405405405405406,
 'trend_ema_fast / Year': 0.05405405405405406,
 'trend_ema_fast / volatility_atr': 0.08108108108108114,
 'trend_ema_fast / volume_nvi': 0.05405405405405406,
 'trend_ema_slow': 0.05405405405405406,
 'trend_ema_slow / Adj Close': 0.0,
 'trend_ema_slow / Month': 0.05405405405405406,
 'trend_ema_slow / Quarter': 0.05405405405405406,
 'trend_ema_slow / Week': 0.05405405405405406,
 'trend_ema_slow / Year': 0.10810810810810811,
 'trend_ema_slow / volatility_atr': 0.08108108108108114,
 'trend_ema_slow / volume_nvi': 0.05405405405405406,
 'trend_ichimoku_a': 0.10810810810810811,
 'trend_ichimoku_a / Month': 0.05405405405405406,
 'trend_ichimoku_a / Quarter': 0.05405405405405406,
 'trend_ichimoku_a / Week': 0.05405405405405406,
 'trend_ichimoku_a / Year': 0.05405405405405406,
 'trend_ichimoku_b': 0.05405405405405406,
 'trend_ichimoku_b / Adj Close': 0.08108108108108114,
 'trend_ichimoku_b / Close': 0.0,
 'trend_ichimoku_b / High': 0.05405405405405406,
 'trend_ichimoku_b / Low': 0.1351351351351352,
 'trend_ichimoku_b / Month': 0.05405405405405406,
 'trend_ichimoku_b / Open': 0.1351351351351352,
 'trend_ichimoku_b / Quarter': 0.05405405405405406,
 'trend_ichimoku_b / Week': 0.05405405405405406,
 'trend_kst': 0.10810810810810811,
 'trend_kst / Adj Close': 0.027027027027027084,
 'trend_kst / Close': 0.05405405405405406,
 'trend_kst / High': 0.05405405405405406,
 'trend_kst / Low': 0.16216216216216217,
 'trend_kst / Month': 0.10810810810810811,
 'trend_kst / Quarter': 0.05405405405405406,
 'trend_kst / Week': 0.05405405405405406,
 'trend_kst / Year': 0.0,
 'trend_kst_sig': 0.05405405405405406,
 'trend_kst_sig / Adj Close': 0.16216216216216217,
 'trend_kst_sig / Close': 0.05405405405405406,
 'trend_kst_sig / High': 0.05405405405405406,
 'trend_kst_sig / Low': 0.08108108108108114,
 'trend_kst_sig / Month': 0.1351351351351352,
 'trend_kst_sig / Open': 0.05405405405405406,
 'trend_kst_sig / Quarter': 0.05405405405405406,
 'trend_kst_sig / Week': 0.05405405405405406,
 'trend_kst_sig / Year': 0.05405405405405406,
 'trend_macd': 0.0,
 'trend_macd / Open': 0.16216216216216217,
 'trend_macd / Quarter': 0.05405405405405406,
 'trend_macd / Week': 0.05405405405405406,
 'trend_macd / Year': 0.1351351351351352,
 'trend_macd / volatility_atr': 0.027027027027027084,
 'trend_macd / volatility_bbhi': 0.08108108108108114,
 'trend_macd / volatility_bbli': 0.10810810810810811,
 'trend_macd / volume_nvi': 0.0,
 'trend_macd_signal': 0.10810810810810811,
 'trend_macd_signal / Adj Close': 0.1351351351351352,
 'trend_macd_signal / Month': 0.08108108108108114,
 'trend_macd_signal / Quarter': 0.05405405405405406,
 'trend_macd_signal / Week': 0.05405405405405406,
 'trend_macd_signal / Year': 0.10810810810810811,
 'trend_macd_signal / volatility_atr': 0.08108108108108114,
 'trend_macd_signal / volatility_bbhi': -0.027027027027026973,
 'trend_macd_signal / volatility_bbl': 0.10810810810810811,
 'trend_macd_signal / volatility_bbm': 0.10810810810810811,
 'trend_mass_index': 0.1351351351351352,
 'trend_mass_index / High': 0.05405405405405406,
 'trend_mass_index / Quarter': 0.05405405405405406,
 'trend_mass_index / Week': 0.05405405405405406,
 'trend_mass_index / Year': 0.027027027027027084,
 'trend_trix': 0.18918918918918926,
 'trend_trix / Month': 0.027027027027027084,
 'trend_trix / Open': 0.027027027027027084,
 'trend_trix / Quarter': 0.05405405405405406,
 'trend_trix / Week': 0.05405405405405406,
 'trend_visual_ichimoku_a': 0.16216216216216217,
 'trend_visual_ichimoku_a / Adj Close': 0.08108108108108114,
 'trend_visual_ichimoku_a / Close': 0.18918918918918926,
 'trend_visual_ichimoku_a / High': 0.027027027027027084,
 'trend_visual_ichimoku_a / Low': 0.18918918918918926,
 'trend_visual_ichimoku_a / Month': 0.08108108108108114,
 'trend_visual_ichimoku_a / Open': 0.0,
 'trend_visual_ichimoku_a / Quarter': 0.05405405405405406,
 'trend_visual_ichimoku_a / Week': 0.05405405405405406,
 'trend_visual_ichimoku_a / Year': 0.05405405405405406,
 'trend_visual_ichimoku_b': 0.16216216216216217,
 'trend_visual_ichimoku_b / Adj Close': 0.10810810810810811,
 'trend_visual_ichimoku_b / Close': 0.027027027027027084,
 'trend_visual_ichimoku_b / High': 0.05405405405405406,
 'trend_visual_ichimoku_b / Low': 0.05405405405405406,
 'trend_visual_ichimoku_b / Month': 0.08108108108108114,
 'trend_visual_ichimoku_b / Open': 0.10810810810810811,
 'trend_visual_ichimoku_b / Quarter': 0.05405405405405406,
 'trend_visual_ichimoku_b / Week': 0.05405405405405406,
 'trend_visual_ichimoku_b / Year': 0.05405405405405406,
 'volatility_atr / Close': 0.05405405405405406,
 'volatility_atr / Low': 0.1351351351351352,
 'volatility_atr / Month': 0.16216216216216217,
 'volatility_atr / Open': 0.0,
 'volatility_atr / Quarter': 0.05405405405405406,
 'volatility_atr / Week': 0.05405405405405406,
 'volatility_atr / trend_ema_slow': 0.16216216216216217,
 'volatility_atr / volatility_bbh': 0.21621621621621623,
 'volatility_atr / volatility_bbhi': 0.08108108108108114,
 'volatility_atr / volatility_bbl': 0.05405405405405406,
 'volatility_atr / volatility_bbli': 0.18918918918918926,
 'volatility_atr / volatility_bbm': 0.05405405405405406,
 'volatility_atr / volatility_dch': 0.05405405405405406,
 'volatility_atr / volatility_dchi': 0.1351351351351352,
 'volatility_atr / volatility_dcl': 0.10810810810810811,
 'volatility_atr / volatility_dcli': 0.05405405405405406,
 'volatility_atr / volatility_kcc': 0.10810810810810811,
 'volatility_atr / volatility_kch': 0.05405405405405406,
 'volatility_atr / volatility_kcl': 0.05405405405405406,
 'volatility_atr / volume_nvi': 0.05405405405405406,
 'volatility_bbh / Adj Close': 0.21621621621621623,
 'volatility_bbh / Month': 0.16216216216216217,
 'volatility_bbh / Quarter': 0.05405405405405406,
 'volatility_bbh / Week': 0.05405405405405406,
 'volatility_bbh / Year': 0.08108108108108114,
 'volatility_bbh / volatility_atr': 0.05405405405405406,
 'volatility_bbh / volatility_bbhi': 0.10810810810810811,
 'volatility_bbh / volatility_bbli': 0.05405405405405406,
 'volatility_bbh / volatility_dcli': 0.05405405405405406,
 'volatility_bbh / volume_nvi': 0.027027027027027084,
 'volatility_bbl': 0.05405405405405406,
 'volatility_bbl / Month': 0.05405405405405406,
 'volatility_bbl / Quarter': 0.05405405405405406,
 'volatility_bbl / Week': 0.05405405405405406,
 'volatility_bbl / Year': 0.05405405405405406,
 'volatility_bbl / volatility_atr': 0.08108108108108114,
 'volatility_bbl / volatility_bbhi': 0.05405405405405406,
 'volatility_bbl / volatility_bbli': 0.027027027027027084,
 'volatility_bbl / volatility_dchi': 0.05405405405405406,
 'volatility_bbl / volatility_dcli': -0.027027027027026973,
 'volatility_bbl / volume_nvi': 0.05405405405405406,
 'volatility_bbm': 0.05405405405405406,
 'volatility_bbm / Month': 0.1351351351351352,
 'volatility_bbm / Quarter': 0.05405405405405406,
 'volatility_bbm / Week': 0.05405405405405406,
 'volatility_bbm / Year': 0.05405405405405406,
 'volatility_bbm / volatility_atr': 0.05405405405405406,
 'volatility_bbm / volatility_bbhi': 0.05405405405405406,
 'volatility_bbm / volatility_bbli': 0.05405405405405406,
 'volatility_bbm / volatility_dchi': 0.05405405405405406,
 'volatility_bbm / volatility_dcli': 0.0,
 'volatility_bbm / volume_nvi': 0.05405405405405406,
 'volatility_dch / Month': 0.027027027027027084,
 'volatility_dch / Quarter': 0.05405405405405406,
 'volatility_dch / Week': 0.05405405405405406,
 'volatility_dch / Year': 0.05405405405405406,
 'volatility_dch / volatility_atr': 0.05405405405405406,
 'volatility_dch / volatility_bbhi': 0.05405405405405406,
 'volatility_dch / volatility_bbli': 0.05405405405405406,
 'volatility_dch / volatility_kcc': -0.027027027027026973,
 'volatility_dch / volume_nvi': 0.05405405405405406,
 'volatility_dchi / volatility_bbhi': 0.08108108108108114,
 'volatility_dcl': 0.05405405405405406,
 'volatility_dcl / Month': 0.05405405405405406,
 'volatility_dcl / Quarter': 0.05405405405405406,
 'volatility_dcl / Week': 0.05405405405405406,
 'volatility_dcl / Year': 0.05405405405405406,
 'volatility_dcl / volatility_atr': 0.08108108108108114,
 'volatility_dcl / volatility_bbhi': 0.05405405405405406,
 'volatility_dcl / volatility_bbli': 0.05405405405405406,
 'volatility_dcl / volume_nvi': 0.0,
 'volatility_kcc': 0.05405405405405406,
 'volatility_kcc / Month': 0.05405405405405406,
 'volatility_kcc / Quarter': 0.05405405405405406,
 'volatility_kcc / Week': 0.05405405405405406,
 'volatility_kcc / Year': 0.05405405405405406,
 'volatility_kcc / volatility_atr': 0.05405405405405406,
 'volatility_kcc / volatility_bbhi': 0.05405405405405406,
 'volatility_kcc / volatility_bbli': 0.05405405405405406,
 'volatility_kcc / volatility_dchi': 0.16216216216216217,
 'volatility_kcc / volatility_dcli': 0.05405405405405406,
 'volatility_kcc / volume_nvi': 0.05405405405405406,
 'volatility_kch': 0.1351351351351352,
 'volatility_kch / Month': 0.10810810810810811,
 'volatility_kch / Quarter': 0.05405405405405406,
 'volatility_kch / Week': 0.05405405405405406,
 'volatility_kch / Year': 0.08108108108108114,
 'volatility_kch / volatility_atr': 0.05405405405405406,
 'volatility_kch / volatility_bbh': -0.027027027027026973,
 'volatility_kch / volatility_bbhi': 0.05405405405405406,
 'volatility_kch / volatility_bbli': 0.05405405405405406,
 'volatility_kch / volume_nvi': 0.027027027027027084,
 'volatility_kcl': 0.05405405405405406,
 'volatility_kcl / Month': 0.05405405405405406,
 'volatility_kcl / Quarter': 0.05405405405405406,
 'volatility_kcl / Week': 0.05405405405405406,
 'volatility_kcl / Year': 0.05405405405405406,
 'volatility_kcl / volatility_atr': 0.05405405405405406,
 'volatility_kcl / volatility_bbhi': 0.05405405405405406,
 'volatility_kcl / volume_nvi': 0.05405405405405406,
 'volume_adi / volatility_bbhi': 0.0,
 'volume_adi / volatility_bbli': 0.027027027027027084,
 'volume_adi / volatility_dchi': 0.0,
 'volume_adi / volatility_dcli': 0.0,
 'volume_cmf / trend_macd': 0.10810810810810811,
 'volume_cmf / trend_macd_signal': 0.16216216216216217,
 'volume_cmf / volatility_bbhi': 0.10810810810810811,
 'volume_cmf / volatility_bbli': 0.027027027027027084,
 'volume_cmf / volatility_dchi': 0.10810810810810811,
 'volume_cmf / volatility_dcli': 0.0,
 'volume_fi / volatility_bbhi': 0.027027027027027084,
 'volume_fi / volatility_bbli': 0.18918918918918926,
 'volume_fi / volatility_dchi': 0.05405405405405406,
 'volume_fi / volatility_dcli': 0.027027027027027084,
 'volume_nvi': 0.05405405405405406,
 'volume_nvi / Adj Close': 0.027027027027027084,
 'volume_nvi / Close': 0.21621621621621623,
 'volume_nvi / High': 0.08108108108108114,
 'volume_nvi / Low': 0.05405405405405406,
 'volume_nvi / Month': 0.05405405405405406,
 'volume_nvi / Open': 0.10810810810810811,
 'volume_nvi / Quarter': 0.05405405405405406,
 'volume_nvi / Week': 0.05405405405405406,
 'volume_nvi / Year': 0.05405405405405406,
 'volume_nvi / trend_ema_fast': 0.05405405405405406,
 'volume_nvi / trend_ema_slow': 0.05405405405405406,
 'volume_nvi / volatility_atr': 0.0,
 'volume_nvi / volatility_bbh': 0.10810810810810811,
 'volume_nvi / volatility_bbhi': 0.0,
 'volume_nvi / volatility_bbl': 0.05405405405405406,
 'volume_nvi / volatility_bbli': 0.05405405405405406,
 'volume_nvi / volatility_bbm': 0.05405405405405406,
 'volume_nvi / volatility_dch': 0.16216216216216217,
 'volume_nvi / volatility_dchi': 0.05405405405405406,
 'volume_nvi / volatility_dcl': 0.1351351351351352,
 'volume_nvi / volatility_dcli': 0.05405405405405406,
 'volume_nvi / volatility_kcc': 0.05405405405405406,
 'volume_nvi / volatility_kch': 0.05405405405405406,
 'volume_nvi / volatility_kcl': 0.05405405405405406,
 'volume_obv': 0.0,
 'volume_obv / Adj Close': 0.0,
 'volume_obv / Close': 0.05405405405405406,
 'volume_obv / High': 0.05405405405405406,
 'volume_obv / High - Low': 0.18918918918918926,
 'volume_obv / Low': 0.08108108108108114,
 'volume_obv / Month': 0.10810810810810811,
 'volume_obv / Open': 0.16216216216216217,
 'volume_obv / Quarter': 0.05405405405405406,
 'volume_obv / Week': 0.05405405405405406,
 'volume_obv / Year': 0.027027027027027084,
 'volume_obv / trend_ema_fast': 0.1351351351351352,
 'volume_obv / trend_ema_slow': 0.027027027027027084,
 'volume_obv / trend_mass_index': 0.1351351351351352,
 'volume_obv / trend_vortex_ind_neg': 0.0,
 'volume_obv / volatility_atr': 0.0,
 'volume_obv / volatility_bbh': 0.1351351351351352,
 'volume_obv / volatility_bbhi': 0.05405405405405406,
 'volume_obv / volatility_bbl': -0.027027027027026973,
 'volume_obv / volatility_bbli': 0.1351351351351352,
 'volume_obv / volatility_bbm': 0.05405405405405406,
 'volume_obv / volatility_dch': 0.1351351351351352,
 'volume_obv / volatility_dchi': -0.027027027027026973,
 'volume_obv / volatility_dcl': 0.05405405405405406,
 'volume_obv / volatility_dcli': 0.0,
 'volume_obv / volatility_kcc': 0.05405405405405406,
 'volume_obv / volatility_kch': 0.05405405405405406,
 'volume_obv / volatility_kcl': 0.05405405405405406,
 'volume_obv / volume_nvi': 0.21621621621621623,
 'volume_vpt / volatility_bbhi': -0.027027027027026973,
 'volume_vpt / volatility_bbli': 0.16216216216216217,
 'volume_vpt / volatility_dcli': 0.05405405405405406}



In [24]:
sorted(feature_scores, reverse=True)

['volume_vpt / volatility_dcli',
 'volume_vpt / volatility_bbli',
 'volume_vpt / volatility_bbhi',
 'volume_obv / volume_nvi',
 'volume_obv / volatility_kcl',
 'volume_obv / volatility_kch',
 'volume_obv / volatility_kcc',
 'volume_obv / volatility_dcli',
 'volume_obv / volatility_dcl',
 'volume_obv / volatility_dchi',
 'volume_obv / volatility_dch',
 'volume_obv / volatility_bbm',
 'volume_obv / volatility_bbli',
 'volume_obv / volatility_bbl',
 'volume_obv / volatility_bbhi',
 'volume_obv / volatility_bbh',
 'volume_obv / volatility_atr',
 'volume_obv / trend_vortex_ind_neg',
 'volume_obv / trend_mass_index',
 'volume_obv / trend_ema_slow',
 'volume_obv / trend_ema_fast',
 'volume_obv / Year',
 'volume_obv / Week',
 'volume_obv / Quarter',
 'volume_obv / Open',
 'volume_obv / Month',
 'volume_obv / Low',
 'volume_obv / High - Low',
 'volume_obv / High',
 'volume_obv / Close',
 'volume_obv / Adj Close',
 'volume_obv',
 'volume_nvi / volatility_kcl',
 'volume_nvi / volatility_kch',
 'v

In [26]:
imp = pd.DataFrame({'col': list(feature_scores.keys()), 'imp': list(feature_scores.values())}).sort_values(by='imp', ascending=False)
imp.head()

,col,imp
495,volume_obv / volume_nvi,0.2162
444,volume_nvi / Close,0.2162
335,volatility_atr / vola...,0.2162
348,volatility_bbh / Adj ...,0.2162
49,High - Low as % of Cl...,0.1892


In [27]:
imp.tail()

,col,imp
29,Day of Week / volatil...,-0.0270
168,Volume / volatility_bbhi,-0.0270
12,Adj Close / volatilit...,-0.0541
23,Close / trend_visual_...,-0.0541
8,Adj Close / trend_vis...,-0.0541


In [0]:
features = imp['col'].tolist()

In [29]:
clf = ''
clf = h2o4gpu.RandomForestClassifier(n_estimators=800,n_jobs=-1, random_state=42)

clf.fit(X_train[features[-6:]], y_train)
ypred = clf.predict(X_test[features[-6:]])
accuracy_score(y_test, ypred)

0.972972972972973

In [30]:
features[-6:]

['Low / trend_mass_index',
 'Day of Week / volatility_bbli',
 'Volume / volatility_bbhi',
 'Adj Close / volatility_dchi',
 'Close / trend_visual_ichimoku_b',
 'Adj Close / trend_visual_ichimoku_a']

In [0]:
features_6 = ['Low / trend_mass_index',
 'Day of Week / volatility_bbli',
 'Volume / volatility_bbhi',
 'Adj Close / volatility_dchi',
 'Close / trend_visual_ichimoku_b',
 'Adj Close / trend_visual_ichimoku_a']

# let's run through a simulation

In [31]:
spy = web.DataReader("SPY", 'yahoo').reset_index()
target = get_target(spy, days_out)
spy = wrangle(spy, False)
spy.shape

/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: divide by zero encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:170: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: divide by zero encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:174: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
/usr/local/lib/python3.6/dist-packages/ta/trend.py:176: RuntimeWarning: invalid value encountered in subtract
  dx = 100 * np.abs((dip - din) / (dip + din))


(2421, 2701)

In [32]:
spy = spy.iloc[:days_out]
spy.shape

(2401, 2701)

In [33]:
target = target.iloc[45:days_out]
target.shape

(2401, 7)

In [1]:
features_scores = {}
counter = 0
this_year_pred = []
this_year_true = []
for day in range(100,0, -1):
    start = time.time()
    counter += 1

    test_day = spy.iloc[len(spy) - day]
    test_target = target.iloc[len(spy) - day][target_feature]
    training_days = spy.iloc[:-day - 37]

    y_train = target.iloc[:-day - 37][target_feature]
    this_year_true.append(test_target)


    X_train_trans = training_days[features_6].values
    X_test_trans = test_day[features_6].values.reshape(1,-1)

    clf = ''
    clf = h2o4gpu.GradientBoostingClassifier(n_estimators=800,
                                n_jobs=-1,
                                random_state=42)

    clf.fit(X_train_trans, y_train)

    ypred = clf.predict(X_test_trans).reshape(1)
    this_year_pred.append(ypred[0])
    score = accuracy_score(this_year_true, this_year_pred)
    if score < .80:
        continue
    print('training max date', training_days['Date'].max())
    print('test date', test_day['Date'])
    print('score', accuracy_score(this_year_true, this_year_pred))
    print('majority class', max(y_train.value_counts(normalize=True)))
    print('probability', clf.predict_proba(X_test_trans))
    print('correct?', ypred[0] == test_target)
    print(counter, '/', 100)
    print('time', time.time() - start)
    print('\n')

NameError: ignored

In [0]:
ypred.reshape(1)[0]

False

In [0]:
sorted(features_scores, reverse=True)

In [0]:
features_scores[sorted(features_scores)[-1]]

In [0]:
features[features_scores[sorted(features_scores)[-1]]]

In [0]:
y_test.value_counts(normalize=True)